In [502]:
import pickle
import numpy as np
from datasets import load_dataset
import tqdm
import math
import sys

In [503]:
# # GLUE
# dataset = load_dataset("glue", "sst2")
# val_dataset = dataset["validation"]

dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

### CNN Based Classifier

In [602]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [603]:
# Define the CNN architecture for 1D vectors
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels = 1, out_channels = 32, kernel_size = 3)
#         self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3)
#         self.conv3 = nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3)
#         self.pool = nn.MaxPool1d(kernel_size = 2)
        self.fc1 = nn.Linear(768, 4)  # Adjust input size according to your data
        self.fc2 = nn.Linear(4, 1)      # Output layer with 1 neuron for binary classification

    def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = self.pool(nn.functional.relu(self.conv3(x)))
#         x = x.view(-1, 16 * 94)  # Adjust the size according to the output size of the last convolutional layer
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

In [604]:
# Sample data loader for demonstration
class SampleDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [605]:
# Get Training Data

# Read pkl file 
with open('embeddings/rotten-MPNET/rotten-train-768.pkl', 'rb') as f:
    vectors = pickle.load(f)
    
X = np.array(vectors)

labels = []
for item in train_dataset:
    labels.append(item['label'])
    
Y = np.array(labels)

### Determining which Subset 

In [667]:
# Read Instance Scores
with open('scores/rotten-MPNET/instance-scores-rotten-L16-30-40-30.pkl', 'rb') as f:
    instance_scores = pickle.load(f)
    
# Sort indices based on the score in descending order
def sort_indices_by_values(values):
    return sorted(range(len(values)), key=lambda i: np.abs(values[i]), reverse = True)

# Example usage:
values = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3]
sorted_indices = sort_indices_by_values(instance_scores)

In [761]:
# Compute subset length

percentage = 0.75
name = 'models/rotten/model-L16-75.pth' 

subset_len = int(percentage * len(X))
sorted_subset_indices = sorted_indices[:subset_len]

In [762]:
X_eff = []
Y_eff = []

for i in sorted_subset_indices:
    X_eff.append(X[i])
    Y_eff.append(Y[i])

In [763]:
N = len(X_eff)
X_train = torch.from_numpy(np.array(X_eff).reshape(N, 1, 768))
Y_train = torch.from_numpy(np.array(Y_eff).reshape(N, 1, 1)).float()
print(str(N) + ' samples selected')

6397 samples selected


In [764]:
c0 = 0
c1 = 0
for i in Y_eff:
    if i == 0:
        c0 += 1
    else:
        c1 += 1
        
print('Class 0: ' + str(c0))
print('Class 1: ' + str(c1))

Class 0: 3190
Class 1: 3207


### Start Training

In [771]:
# Initialize the model, loss function, and optimizer
model = CNNClassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.00005)

# Create data loader
trainset = SampleDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle=True)

# Train the model
for epoch in tqdm.tqdm(range(100)):  # Adjust number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 9:  # Print every 10 mini-batches
            print('[%d, %5d] loss: %.4f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

  2%|▏         | 2/100 [00:00<00:07, 12.31it/s]

[1,    10] loss: 0.0694
[1,   110] loss: 0.7000
[2,    10] loss: 0.0688
[2,   110] loss: 0.6945
[3,    10] loss: 0.0682
[3,   110] loss: 0.6792
[4,    10] loss: 0.0679
[4,   110] loss: 0.6705


  5%|▌         | 5/100 [00:00<00:13,  7.25it/s]

[5,    10] loss: 0.0665
[5,   110] loss: 0.6569
[6,    10] loss: 0.0654


  6%|▌         | 6/100 [00:00<00:13,  7.02it/s]

...

[98,    10] loss: 0.0246
[98,   110] loss: 0.2776


 99%|█████████▉| 99/100 [00:13<00:00,  4.85it/s]

[99,    10] loss: 0.0348
[99,   110] loss: 0.2732


100%|██████████| 100/100 [00:13<00:00,  7.27it/s]

[100,    10] loss: 0.0246
[100,   110] loss: 0.2898
Finished Training


In [772]:
# Save the trained model
torch.save(model.state_dict(), name)

# Perform inference with the saved model
loaded_model = CNNClassifier()
loaded_model.load_state_dict(torch.load(name))
loaded_model.eval()

CNNClassifier(
  (fc1): Linear(in_features=768, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)

In [773]:
# Lets do testing

# Example inference
# sample_input = torch.randn(1, 1, 768)  # Example input
# output = loaded_model(sample_input)
# print("Model output:", output.item())

# Read pkl file 
with open('embeddings/rotten-MPNET/rotten-test-768.pkl', 'rb') as f:
    test_vectors = pickle.load(f)

In [774]:
gt = []
preds = []

for i in tqdm.tqdm(range(len(test_vectors))):
    input_vector = torch.tensor(test_vectors[i].reshape(1, 1, 768))
    actual = test_dataset[i]['label']
    output = loaded_model(input_vector) 
    if output < 0.5:
        pred = 0
    else:
        pred = 1
    gt.append(actual)
    preds.append(pred)

100%|██████████| 1066/1066 [00:00<00:00, 13721.51it/s]


In [775]:
# Print the number of misclassifications
misc = 0
for i in range(len(gt)):
    if gt[i] != preds[i]:
        misc += 1

print(misc)

159


In [776]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(gt, preds)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       533
           1       0.86      0.84      0.85       533

    accuracy                           0.85      1066
   macro avg       0.85      0.85      0.85      1066
weighted avg       0.85      0.85      0.85      1066

